<a href="https://colab.research.google.com/github/maetthil/NASASpaceApps/blob/main/Read_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # %%capture
# !pip install cdflib
# !pip install pyspedas
# !pip install matplotlib==3.1.2

In [6]:
import glob
import sys
import os
import re
import requests
import numpy as np
import pytz
import datetime
import pandas as pd
from tqdm import tqdm

import pyspedas
from pytplot import tplot
import pytplot

import matplotlib.pyplot as plt


## Magnetic Field

In [ ]:
def getMagneticField(satellite="wind", trange=['2018-01-01', '2018-01-10']):
    pytplot.del_data()
    if satellite == "wind":
        vars = pyspedas.wind.mfi(trange=trange)
        quants_mag = pytplot.data_quants['BGSE']
        mag = quants_mag.values

        

    else:
        mag = pyspedas.dscovr.mag(trange=trange)
        quants_mag = pytplot.data_quants['dsc_h0_mag_B1GSE']
        mag = quants_mag.values
        mag[mag == -1e+31] = np.nan

    

    time_mag = quants_mag.coords['time'].values
    time_mag = [datetime.datetime.fromtimestamp(i, pytz.timezone("UTC")) for i in time_mag]

    df_mag = pd.DataFrame({"Bx_GSE": mag[:,0], 
                            "By_GSE": mag[:,1],
                            "Bz_GSE": mag[:,2]}, index = time_mag)
    
    return df_mag

## Ion Plasma

In [ ]:
def getIonPlasma(satellite="wind", trange=['2018-01-01', '2018-01-10']):
    pytplot.del_data()
    if satellite == "wind":
        vars = pyspedas.wind.swe(trange=trange)
        quants_vGSE = pytplot.data_quants['U_eGSE']
        quants_therSp = pytplot.data_quants['T_elec']
        quants_Np = pytplot.data_quants['N_elec']

        vGSE = quants_vGSE.values
        therSp = quants_therSp.values
        Np = quants_Np.values

        

    else:
        vars = pyspedas.dscovr.fc(trange=trange)

        quants_vGSE = pytplot.data_quants['dsc_h1_fc_V_GSE']
        quants_therSp = pytplot.data_quants['dsc_h1_fc_THERMAL_SPD']
        quants_Np = pytplot.data_quants['dsc_h1_fc_Np']

        vGSE = quants_vGSE.values
        therSp = quants_therSp.values
        Np = quants_Np.values

        vGSE[vGSE == -1e+31] = np.nan
        therSp[therSp == -1e+31] = np.nan
        Np[Np == -1e+31] = np.nan

    

    time_fc = quants_vGSE.coords['time'].values
    time_fc = [datetime.datetime.fromtimestamp(i, pytz.timezone("UTC")) for i in time_fc]

    df_ions = pd.DataFrame({"Vx_GSE": vGSE[:,0], 
                             "Vy_GSE": vGSE[:,1],
                             "Vz_GSE": vGSE[:,2],
                             "Therm_spd": therSp,
                             "Np":Np }, index = time_fc)
    
    return df_ions

In [39]:
ions = getIonPlasma(satellite="dscovr", trange=['2018-01-01', '2018-01-10'])
ions.head()

01-Oct-22 17:17:05: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/dscovr/h1/faraday_cup/2018/
01-Oct-22 17:17:05: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180101_v08.cdf
01-Oct-22 17:17:06: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180102_v08.cdf
01-Oct-22 17:17:06: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180103_v08.cdf
01-Oct-22 17:17:06: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180105_v08.cdf
01-Oct-22 17:17:06: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180106_v08.cdf
01-Oct-22 17:17:06: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180107_v08.cdf
01-Oct-22 17:17:06: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180108_v08.cdf
01-Oct-22 17:17:07: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180109_v08.cdf


,Vx_GSE,Vy_GSE,Vz_GSE,Therm_spd,Np
2018-01-01 00:00:00+00:00,-403.007935,-12.879808,-25.402237,39.441380,14.758022
2018-01-01 00:01:00+00:00,-403.619720,-16.166252,-26.591558,38.992432,14.678254
2018-01-01 00:02:00+00:00,-405.910919,-14.688877,-29.482092,36.989517,14.709406
2018-01-01 00:03:00+00:00,-404.753052,-13.926281,-30.157286,36.591244,14.661954
2018-01-01 00:04:00+00:00,-403.666260,-13.622793,-28.274202,36.837086,14.399885


In [38]:
mag = getMagneticField(satellite="dscovr", trange=['2018-01-01', '2018-01-10'])
mag.head()

01-Oct-22 17:17:00: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/dscovr/h0/mag/2018/
01-Oct-22 17:17:01: File is current: dscovr_data/h0/mag/2018/dscovr_h0_mag_20180101_v01.cdf
01-Oct-22 17:17:01: File is current: dscovr_data/h0/mag/2018/dscovr_h0_mag_20180102_v01.cdf
01-Oct-22 17:17:01: File is current: dscovr_data/h0/mag/2018/dscovr_h0_mag_20180103_v01.cdf
01-Oct-22 17:17:01: File is current: dscovr_data/h0/mag/2018/dscovr_h0_mag_20180104_v01.cdf
01-Oct-22 17:17:01: File is current: dscovr_data/h0/mag/2018/dscovr_h0_mag_20180105_v01.cdf
01-Oct-22 17:17:01: File is current: dscovr_data/h0/mag/2018/dscovr_h0_mag_20180106_v01.cdf
01-Oct-22 17:17:01: File is current: dscovr_data/h0/mag/2018/dscovr_h0_mag_20180107_v01.cdf
01-Oct-22 17:17:02: File is current: dscovr_data/h0/mag/2018/dscovr_h0_mag_20180108_v01.cdf
01-Oct-22 17:17:02: File is current: dscovr_data/h0/mag/2018/dscovr_h0_mag_20180109_v01.cdf


,Bx_GSE,By_GSE,Bz_GSE
2018-01-01 00:00:00.500000+00:00,5.837182,2.314090,-7.158201
2018-01-01 00:00:01.500000+00:00,5.899103,2.184921,-7.075517
2018-01-01 00:00:02.500000+00:00,6.042387,2.221216,-7.145005
2018-01-01 00:00:03.500000+00:00,5.720763,2.225209,-6.768812
2018-01-01 00:00:04.500000+00:00,5.578448,2.175153,-6.790361
